# WeatherPy

## Imports

In [1]:
from citipy import citipy
import numpy as np
import pandas as pd
import json 
import requests
import matplotlib.pyplot as plt
from api_keys import weather_api_key
from api_keys import g_key

## Generate Cities List

In [2]:
#First want to create my list of 500 lat/lng pairs
# it says AT LEAST 500 so how many?????
# assuming best way is to obtain from a dataframe like we did in examples
# Lat and Long: in decimal degrees format and range from -90 to 90 for latitude and -180 to 180 for longitude.
# so for randomness will need to go through whole range of lats and whole range of longs
# also need to make sure they are unique pairs.
# could I make massive list and remove duplicates?? or is there a unique random function

# don't want integers for lat/long b/c need decimals
# numpy.random.uniform(low=0.0, high=1.0, size=None) seems best
# everything else only goes b/w 0 and 1 for numpy.random and other random features
# also seems you could use  import random   random.uniform(a, b)
# round(random.uniform(1,2), N) to ensure decimals

# also, how many decimal places do lat/long need. I saw somehting that said 15
# I wonder if there is a requirement for the google thingo and if I can round

from numpy.random import seed


# this sets it so that everytime I run seed(1) and then the random thingo it will give the same random numbers
# just to help me get a handle on what's happening
# works for both
seed(1)

lat_number = np.random.uniform(-90, 90, 1300)

long_number = np.random.uniform(-180, 180, 1300)

#print(lat_number)
#print(long_number)

In [3]:
# make my data frame out of these lists

number_df = pd.DataFrame({"Latitude":lat_number, "Longitude": long_number})
number_df.head()

,Latitude,Longitude
0,-14.936039,13.322562
1,39.658409,-48.245995
2,-89.979413,37.550464
3,-35.580137,165.168906
4,-63.583940,-38.583740


In [4]:
#DON'T NEED
# dropping any duplicate for a subset of the two columns to ensure unique values
#number_df = number_df.drop_duplicates(subset=['Latitude', 'Longitude'])
#number_df

In [5]:
# this also check is dupicate rows but there aren't any
#duplicate_rows = number_df[number_df.duplicated(['Latitude', 'Longitude'])]
#duplicate_rows

In [6]:
#Now to use citipy
# will need to loop through the lats and longs find a city for each, and then put these cities into a list

# setting enmpty city list
cities = []

# citipy finds country code as well
# do I want that???
# are there cities with the same name across the world or??
# I'll put it in for now and then see
country = []


# itterating through each row using the index
for x in number_df.index:
    lat = number_df['Latitude'][x]
    long = number_df['Longitude'][x]
    city = citipy.nearest_city(lat, long)
    city_name = city.city_name
    country_code = city.country_code
    
    cities.append(city_name)
    country.append(country_code)
    
# do I then just add them to my dataframe??

number_df["City"] = cities
number_df["Country"] = country
number_df

#>>> city = citipy.nearest_city(22.99, 120.21)
#>>> city
#<citipy.City instance at 0x1069b6518>
#>>>
#>>> city.city_name     # Tainan, my home town
#'tainan'
#>>>
#>>> city.country_code
#'tw'                  # And the country is surely Taiwan


#Randomly select at least 500 unique (non-repeat) cities based on latitude and longitude.
#The city data you generate is based on random coordinates as well as different query times; as such, 
#your outputs will not be an exact match to the provided starter notebook.

# what is a query time?
# i think it is time taken for requested info to return
# so I don't think it is something I need to worry about 

,Latitude,Longitude,City,Country
0,-14.936039,13.322562,lubango,ao
1,39.658409,-48.245995,torbay,ca
2,-89.979413,37.550464,port elizabeth,za
3,-35.580137,165.168906,ahipara,nz
4,-63.583940,-38.583740,ushuaia,ar
...,...,...,...,...
1295,56.959632,1.215939,boddam,gb
1296,-2.929149,9.777663,gamba,ga
1297,16.191234,-76.291348,morant bay,jm
1298,51.707421,4.319723,steenbergen,nl


In [7]:
# should I actually just check if any city names are the same??? becuase that is what I'm searching for right
# and if any are the same then same answer???

duplicate_rows = number_df[number_df.duplicated(subset = ['City'])]
duplicate_rows

# this is better because now I'm finding the places with duplicate city names, because obvs multiple different lats/lng
# can give the same city
# so don't need above part removing lat/lng dupliates just this bit probs
# actually the bit below this is just me checking

,Latitude,Longitude,City,Country
5,-73.379053,-38.529721,ushuaia,ar
18,-64.730351,88.796690,busselton,au
27,-82.970139,-94.901724,punta arenas,cl
28,-59.430524,57.815408,taolanaro,mg
30,-72.297570,-104.122461,punta arenas,cl
...,...,...,...,...
1290,86.866162,85.610656,dikson,ru
1291,74.473132,-81.852379,qaanaaq,gl
1292,-27.473504,-6.665108,jamestown,sh
1296,-2.929149,9.777663,gamba,ga


In [8]:
# dropping dupicate cities

number_df = number_df.drop_duplicates(subset=['City'])
number_df

# default sets as keep = first I think

# end up with 572 rows!!

# I think I should redo the index so that the numbers aren't gappy now that I've deleted the duplicates

,Latitude,Longitude,City,Country
0,-14.936039,13.322562,lubango,ao
1,39.658409,-48.245995,torbay,ca
2,-89.979413,37.550464,port elizabeth,za
3,-35.580137,165.168906,ahipara,nz
4,-63.583940,-38.583740,ushuaia,ar
...,...,...,...,...
1293,50.440424,-110.868744,medicine hat,ca
1294,8.361826,124.960650,maluko,ph
1295,56.959632,1.215939,boddam,gb
1297,16.191234,-76.291348,morant bay,jm


In [10]:
# Now need to do API calls in a loop, but a series of api calls becuase think can only do 10 at once or something
# in the example they created the new rows on the dataframe first so that within the loop they just add data to it.
# following example NearestRestr
# need Temperature, Humidity, Cloudiness, Wind Speed (mph)
# df in starter code also has date but I don't know what this is

cities_df = number_df.copy()

cities_df['Temperature'] = ""
cities_df['Humidity'] = ""
cities_df['Cloudiness'] = ""
cities_df['Wind Speed (mph)'] = ""
cities_df['Date'] = ""

cities_df.head()


,Latitude,Longitude,City,Country,Temperature,Humidity,Cloudiness,Wind Speed (mph),Date
0,-14.936039,13.322562,lubango,ao,,,,,
1,39.658409,-48.245995,torbay,ca,,,,,
2,-89.979413,37.550464,port elizabeth,za,,,,,
3,-35.580137,165.168906,ahipara,nz,,,,,
4,-63.583940,-38.583740,ushuaia,ar,,,,,


In [11]:
#doing json thing to see what it is like
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

#above is base url

# Build partial query URL
query_url = f"{url}appid={weather_api_key}&units={units}&q="
city = 'melbourne'

response = requests.get(query_url + city).json()

print(json.dumps(response, indent = 4, sort_keys = True))

print (response["dt"])

{
    "base": "stations",
    "clouds": {
        "all": 1
    },
    "cod": 200,
    "coord": {
        "lat": 28.0836,
        "lon": -80.6081
    },
    "dt": 1610345291,
    "id": 4163971,
    "main": {
        "feels_like": 11.87,
        "humidity": 100,
        "pressure": 1023,
        "temp": 12.25,
        "temp_max": 14,
        "temp_min": 11.67
    },
    "name": "Melbourne",
    "sys": {
        "country": "US",
        "id": 4922,
        "sunrise": 1610367315,
        "sunset": 1610405116,
        "type": 1
    },
    "timezone": -18000,
    "visibility": 10000,
    "weather": [
        {
            "description": "clear sky",
            "icon": "01n",
            "id": 800,
            "main": "Clear"
        }
    ],
    "wind": {
        "deg": 340,
        "speed": 1.54
    }
}
1610345291


In [14]:
# setting up loop

# how do I do this in sets
# example seems to go in sets of 50 

url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

#above is base url

# Build partial query URL
query_url = f"{url}appid={weather_api_key}&units={units}&q="


 # assemble url and make API request
    print("Beginning Data Retrieval")
    print("-------------------------")

# use iterrows to iterate through pandas dataframe
for index, row in cities_df.iterrows():

    # get city from df
    city = row['City']

    response = requests.get(query_url + city).json()

    # adding to data frame using try and except incase no data
    
    try:
        print(f"Retrieving Results for Index {index}: {city}.")
        
        # do I want temp_max?
        cities_df.loc[index, 'Temperature'] = response["main"]["temp"]
        cities_df.loc[index, 'Humidity'] = response["main"]["humidity"]
        cities_df.loc[index, 'Cloudiness'] = response["clouds"]["all"]
        cities_df.loc[index, 'Wind Speed (mph)'] = response["wind"]["speed"]
        cities_df.loc[index, 'Date'] = response["dt"]
       
        
    except (KeyError, IndexError):
        print("City not found... skipping.")
        
    print("------------")
    
    
    
    

#will use something like this for looping and printing

# find the closest restaurant of each type to coordinates

#base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#params = {
    #"location": "39.952583,-75.16522",  # philadelphia coords,
    #"rankby": "distance",
    #"type": "restaurant",
    #"key": gkey,
#}
# use iterrows to iterate through pandas dataframe
#for index, row in types_df.iterrows():

    # get restaurant type from df
    #restr_type = row['ethnicity']

    # add keyword to params dict
    #params['keyword'] = restr_type
    # adding new parameter in the loop becuase it is changing and will be the ethnicity we are up to

    # assemble url and make API request
    #print(f"Retrieving Results for Index {index}: {restr_type}.")
    #response = requests.get(base_url, params=params).json()
    
    # extract results
    #results = response['results']
    
    #try:
        #print(f"Closest {restr_type} restaurant is {results[0]['name']}.")
        
        #types_df.loc[index, 'name'] = results[0]['name']
        #types_df.loc[index, 'address'] = results[0]['vicinity']
        #types_df.loc[index, 'price_level'] = results[0]['price_level']
        #types_df.loc[index, 'rating'] = results[0]['rating']
        
    #except (KeyError, IndexError):
        #print("Missing field/result... skipping.")
        
    #print("------------")



Beginning Data Retrieval
-------------------------
Retrieving Results for Index 0: lubango.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 1: torbay.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 2: port elizabeth.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 3: ahipara.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 4: ushuaia.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 6: busselton.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 7: taolanaro.
City not found... skipping.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 8: caravelas.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 9: kulhudhuffushi.
------------
Beginnin

Retrieving Results for Index 89: naze.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 91: narsaq.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 92: banjar.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 93: chateaubelair.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 94: desaguadero.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 96: pevek.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 97: matameye.
City not found... skipping.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 100: luderitz.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 101: salinopolis.
------------
Beginning Data Retrieval
-------------------------
Retriev

Retrieving Results for Index 210: hualmay.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 212: kaitangata.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 215: puerto ayora.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 218: tigil.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 221: byron bay.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 223: grindavik.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 226: constitucion.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 229: mys shmidta.
City not found... skipping.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 230: victoria.
------------
Beginning Data Retrieval
---------------

Retrieving Results for Index 342: la ronge.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 343: sangar.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 345: tateyama.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 346: saint george.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 347: arlit.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 348: novomikhaylovskiy.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 350: grafton.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 351: praia da vitoria.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 355: avera.
------------
Beginning Data Retrieval
-------------------------
Retrieving Resul

Retrieving Results for Index 510: yanam.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 511: grand river south east.
City not found... skipping.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 514: el faiyum.
City not found... skipping.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 515: booue.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 516: mongoumba.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 518: porto novo.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 520: sioux lookout.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 521: salalah.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 524: khammam.
------------
Beginnin

Retrieving Results for Index 684: sao joao da barra.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 685: barraute.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 686: hami.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 687: paraiso.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 691: dingle.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 692: oksfjord.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 703: lebu.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 704: mancio lima.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 705: luanda.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 

Retrieving Results for Index 879: hendijan.
City not found... skipping.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 880: henties bay.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 883: kaeo.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 884: harlingen.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 885: harper.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 891: inhambupe.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 895: walvis bay.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 897: kyabe.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 899: krugersdorp.
------------
Beginning Data Retrieval
-------------------------

Retrieving Results for Index 1123: crab hill.
City not found... skipping.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 1127: conceicao do araguaia.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 1128: bereda.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 1131: saint-leu.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 1133: coahuayana.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 1138: prince rupert.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 1145: azad shahr.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 1146: coronado.
------------
Beginning Data Retrieval
-------------------------
Retrieving Results for Index 1147: avenal.
------------
Beginning Data Retriev

In [15]:
# want to export to csv

#cities_df.to_csv("city_weather_data.csv", encoding="utf-8", index=False)


cities_df

,Latitude,Longitude,City,Country,Temperature,Humidity,Cloudiness,Wind Speed (mph),Date
0,-14.936039,13.322562,lubango,ao,14.72,73,100,0.94,1610345881
1,39.658409,-48.245995,torbay,ca,-0.83,93,90,9.77,1610345892
2,-89.979413,37.550464,port elizabeth,za,21,73,90,4.63,1610345892
3,-35.580137,165.168906,ahipara,nz,19.65,66,6,4.15,1610345892
4,-63.583940,-38.583740,ushuaia,ar,5,65,75,9.77,1610345887
...,...,...,...,...,...,...,...,...,...
1293,50.440424,-110.868744,medicine hat,ca,2.21,60,1,7.72,1610345968
1294,8.361826,124.960650,maluko,ph,26.11,89,100,4.02,1610346034
1295,56.959632,1.215939,boddam,gb,7.67,76,100,5.14,1610346034
1297,16.191234,-76.291348,morant bay,jm,24.52,73,75,2.06,1610346034


In [20]:
cities_df.describe()
# for some reaosn this is only doing it for lat and long column

,Latitude,Longitude
count,572.000000,572.000000
mean,19.401821,14.207347
std,36.958569,92.807436
min,-89.979413,-179.628144
25%,-7.507964,-61.192644
50%,20.605475,15.855764
75%,49.287249,89.004057
max,89.518113,179.467383


In [21]:
cities_df.dtypes

# because objects.

Latitude            float64
Longitude           float64
City                 object
Country              object
Temperature          object
Humidity             object
Cloudiness           object
Wind Speed (mph)     object
Date                 object
dtype: object

In [23]:
# need to convert

cities_df[['Temperature', 'Humidity', 'Cloudiness', 'Wind Speed (mph)', 'Date' ]] = pd.to_numeric(cities_df[['Temperature', 'Humidity', 'Cloudiness', 'Wind Speed (mph)', 'Date']])

TypeError: arg must be a list, tuple, 1-d array, or Series

In [ ]:
# convert all columns of DataFrame
df = df.apply(pd.to_numeric) # convert all columns of DataFrame

# convert just columns "a" and "b"
df[["a", "b"]] = df[["a", "b"]].apply(pd.to_numeric)